# nbjob - Using Jupyter notebooks for machine learning research
*Implements experiment management and lightweight version control.*

* Home: http://nikitakitaev.com/nbjob
* GitHub: http://github.com/nikitakit/nbjob

`nbjob` is a Python library that enhances Jupyter notebooks.

Features:
* Dispatch jobs from Jupyter Notebooks to an ipyparallel cluster
* Designed for long-running jobs, such as training a neural network for greater than a day
* Store intermediate and final job results
* Keep reproducible source code backups for all jobs
* Run analyses on job results
* Easily run a series of jobs related jobs with different hyperparameters

## Contents
* [Background](background.html)
* [Installation](install.html)
* [Tutorial](tutorial.html)